In [96]:
city = ["New York", "San Francisco", "Boston", "Washington D.C.", "Philadelphia",
    "Chicago", "Miami", "Baltimore", "Minneapolis", "Seattle", "Madison"]
city_number = size(city)[1];
city_day_upperlimit = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1];
score = [84, 80, 74, 71, 67, 65, 59, 58, 58, 57, 40];
city_score = Dict(zip(city, score));

raw = readdlm("out.csv",',');
departure_city = raw[:,1];
departure_airport = raw[:2];
arrival_city = raw[:,3];
arrival_airport = raw[:4];
flight_date = raw[:,5];
flight_price = raw[:,6];
flight_comfort = raw[:,7];
flight_number = size(raw)[1];

# max amout of money spend on flight
max_money = 100000;
# max travel days
max_day = 10;

matrix = zeros(2city_number,2city_number + flight_number);
# split each city into two 'subcities'
for i = 1 : city_number
    matrix[2i-1, 2i-1] = 1
    matrix[2i  , 2i-1] = -1
    
    matrix[2i-1, 2i] = -1
    matrix[2i  , 2i] = 1
end
for i = 1 : flight_number
    u = 1
    v = 1
    for j = 1 : city_number
        if departure_city[i] == city[j]
            u = j
        end
        if arrival_city[i] == city[j]
            v = j
        end
    end
    matrix[2u  ,i+2city_number] = 1
    matrix[2v-1,i+2city_number] = -1
end

b = zero(1:2city_number);
b[2city_number-1] = -1;
b[2city_number] = 1;

In [72]:
city_day_upperlimit

1x11 Array{Int32,2}:
 3  3  3  3  3  3  3  3  3  3  3

In [5]:
function getInitialEdge(remaining)
    len = size(remaining)[1];
    for i in 1:len
        edge_no = remaining[i]
        if matrix[22, edge_no] == 1
            return i
        end
    end
    return 1
end

function get_city_from_edge(no)
    arr = matrix[:,no]
    u = 0
    v = 0
    for i = 1 : 2city_number
        if arr[i] == 1
            u = round(Int,ceil(i/2));
        end
        if arr[i] == -1
            v = round(Int,ceil(i/2));
        end
    end
    return u, v
end

# print all the edges, including flight and circle for one city itself
function print_out_edge(edge)
    for i = 1 : 2city_number + flight_number
        if edge[i] > 0
            if i<= 2city_number && i%2 == 1
                continue
            end
            u , v = get_city_from_edge(i)
            print(i, " ", city[u], "  to  ", city[v],'\n')
        end
    end
end

# print all the flights
function print_out_flight(flight)
    for i = 1 : flight_number
        if flight[i] > 0
            print(i,' ',departure_city[i], "  to   ", arrival_city[i], " \n");
        end
    end
end

# HELPER FUNCTION: returns the cycle containing the city START.
function getSubtour(edge_remaining, start_edge)
    subtour = [start_edge];
    start_u, start_v = get_city_from_edge(edge_remaining[start_edge]);

    pre_u = start_u;
    pre_v = start_v;
    while true
        for i = 1 : size(edge_remaining)[1]
            if i in subtour
                continue
            end
            temp_u, temp_v = get_city_from_edge(edge_remaining[i]);
            if temp_u == pre_v
                push!(subtour, i);
                pre_u = temp_u;
                pre_v = temp_v;
                break
            end
        end
        if pre_v == start_u
            break
        end
    end
    return subtour;
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(edge_chosen_list)
    edge_remaining = edge_chosen_list;
    subtours = [];
    while length(edge_remaining) > 0
        initial_edge_in_remaining_list = getInitialEdge(edge_remaining);
        subtour = getSubtour(edge_remaining, initial_edge_in_remaining_list);
        # the subtour contains only the number in remaining list
        # Ex. if remaining list = [ 10 12 4 7], and [10 4] is a cycle, then subtour here is [1 3]
        push!(subtours, edge_remaining[subtour]);
        edge_remaining = setdiff(edge_remaining, edge_remaining[subtour]);
    end
    return subtours;
end


getAllSubtours (generic function with 1 method)

In [99]:
using JuMP

m = Model()
# city that will spend some days to travel
@defVar(m, city_choice[1:city_number], Bin)
# flight that will be chosen
@defVar(m, flight_choice[1:flight_number], Bin)
# edge consists of 11 pairs of split sub-cities, and 320 flights
@defVar(m, edge[1:2city_number + flight_number], Bin)
# time spent on each city
@defVar(m, day_each_city[1:city_number] >= 0, Int)

# first 22 of edges should match the city_choice
@addConstraint(m, c1[i=1:city_number-1], edge[2i-1] == city_choice[i])
# the remaining edges should match the flight
@addConstraint(m, c2[i=1:flight_number], edge[i+2city_number] == flight_choice[i])

# should be a round trip
@addConstraint(m, matrix * edge .== b)
# each point should have at most one out degree and one in degree
@addConstraint(m, abs(matrix) * edge .<= 2)
@addConstraint(m, city_choice[city_number] == 1)
# restriced by money limit
@addConstraint(m, sum{flight_choice[i] * flight_price[i], i=1:flight_number} <= max_money)
# day spent on each city should be less than its upper limit
@addConstraint(m, day_each_city .<= city_day_upperlimit)
@addConstraint(m, day_each_city .>= city_choice)
# restricted by travel day limit
# assumption: spend two days at each city, and one day on each flight
@addConstraint(m,  sum(day_each_city) + sum(flight_choice) <= max_day)

# very important!
# No circle!
# assmpution: each city except Madison is visited only once
#@addConstraint(m, sum(city_choice) == sum(flight_choice))

# try to max the comfortable indexes
@setObjective(m, Max, sum{score[i]*day_each_city[i], i=1:city_number} + sum{flight_comfort[i] * flight_choice[i], i=1:flight_number})

for iter = 1 : 30
    solve(m)
    print("current comfort is :  ", getObjectiveValue(m),'\n')
    print("current day is : ", sum(getValue(day_each_city)) + sum(getValue(flight_choice)),'\n')
    print(getValue(city_choice),'\n')
    print(getValue(day_each_city),'\n')
#    print(getValue(edge[1:22]),'\n')
    edge_choice_exist = getValue(edge)
    edge_chosen_list = []
    for i = 1 : flight_number+2city_number
        if edge_choice_exist[i] > 0
            push!(edge_chosen_list, i)
        end
    end
    subtours = getAllSubtours(edge_chosen_list)
    # subtours is a big map
    for subtour in subtours
        print("subtour :  \n")
        print(subtour,'\n')
        L = length(subtour);
        @addConstraint(m, sum{edge[subtour[i]], i = 1:L} <= L-1)
    end
    print('\n')
    #print_out_tour(getValue(flight_choice))
    print('\n')
    print_out_edge(getValue(edge))
    print('\n')
    print('\n')
    len = length(subtours)
    if len == 1
        print("solved")
        break
    end
end


current comfort is :  788.0
current day is : 10.0
[1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0]
[1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0]
subtour :  
Any[27,1,81]
subtour :  
Any[7,210,11,132,9,271]


27 Madison  to  New York
81 New York  to  Madison
132 Chicago  to  Philadelphia
210 Washington D.C.  to  Chicago
271 Philadelphia  to  Washington D.C.


current comfort is :  788.0
current day is : 10.0
[1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0]
[3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0]
subtour :  
Any[24,3,50]
subtour :  
Any[1,86,11,117]


24 Madison  to  San Francisco
50 San Francisco  to  Madison
86 New York  to  Chicago
117 Chicago  to  New York


current comfort is :  787.0
current day is : 10.0
[1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0]
[3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0]
subtour :  
Any[39,7,210,11,117,1,81]


39 Madison  to  Washington D.C.
81 New York  to  Madison
117 Chicago  to  New York
210 Washington D.C.  to  Chicago


solved